![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b> Chat und Memory </b></font> </br></p>

---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")

!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>
def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']

# 1 | Konversationsverwaltung
---




Große Sprachmodelle (LLMs) ermöglichen eine natürliche Interaktion ähnlich menschlicher Gespräche. Sie sind in der Lage, auf vorherige Informationen innerhalb eines Dialogs Bezug zu nehmen. In diesem Modul werden wir verschiedene Ansätze zur Verwaltung des Gedächtnisses eines LLMs untersuchen.

Da der interne Kontextpuffer eines LLMs begrenzt ist, benötigen wir Strategien, um den Chat-Verlauf zu speichern und effizient zu nutzen. LangChain bietet verschiedene Lösungsansätze, die wir in diesem Modul erkunden werden:



1. **Manuelle Konversationsverwaltung**: Einfache Speicherung und Weiterleitung der kompletten Konversationshistorie
2. **Begrenzung der Nachrichtenanzahl**: Verwendung eines "Fensters" der letzten N Nachrichten
3. **Token-basierte Limitierung**: Berücksichtigung der tatsächlichen Tokenlimits des LLM
4. **Zusammenfassung von Konversationen**: Komprimieren längerer Konversationen
5. **Persistenter Speicher**: Langfristige Speicherung über Sitzungen hinweg

Jeder dieser Ansätze hat seine eigenen Vor- und Nachteile, die wir im Folgenden genauer untersuchen werden.


<p><font color='black' size="5">
Message-Typen
</font></p>

Die memory-Module von LangChain erwarten Message-Objekte wie SystemMessage, HumanMessage, AIMessage.

Sie helfen LangChain (und dem Modell), den Kontext richtig einzuordnen:    
Wer sagt was? Welche Rolle hat diese Nachricht im Dialog?

Das ist besonders wichtig, wenn man mit ConversationBufferMemory, ConversationSummaryMemory usw. arbeitest, weil diese listenweise Messages speichern und später wiederverwenden.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown

In [ ]:
messages = [
    SystemMessage(
        content="Du bist ein kompetenter und hilfreicher Assistent."
    ),
    HumanMessage(
        content="Nenne mir die Hauptstadt von Usbekistan?"
    ),
]

In [ ]:
type(messages)

In [ ]:
MODEL = 'gpt-4o-mini'
TEMPERATURE = 0.0

llm = ChatOpenAI(model=MODEL, temperature= TEMPERATURE)
response = llm.invoke(messages)

In [ ]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

<p><font color='black' size="5">
Manuelle Konversationsverwaltung
</font></p>




Im ersten Beispiel werden wir einen Chatbot schrittweise mit einer manuellen Konversationsverwaltung aufbauen. Dieser Ansatz bietet die volle Kontrolle, indem wir die Konversationshistorie explizit verwalten und weitergeben.

In [ ]:
# Abschnitt 0: Installation und API-Key
!uv pip install --system --prerelease allow -q langchain_community langchain_openai

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [ ]:
# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"
TEMPERATURE = 0.0
MEMORY_WINDOW = 100 # hoher Wert, praktisch keine Beschränkung der Anzahl

DEFAULT_SYSTEM = """
Du bist ein kompetenter KI-Assistent mit breitem Fachwissen.

Deine Antworten sind:
- Klar strukturiert und mit Markdown formatiert
- Praxisorientiert und direkt umsetzbar
- Basierend auf aktuellem Kenntnisstand
- Mit passenden Beispielen versehen
- In verständlicher Sprache formuliert

Bei deiner Arbeit:
- Analysierst du Fragen sorgfältig
- Gibst präzise und relevante Antworten
- Erkennst den Kontext der Anfrage
- Bietest bei Bedarf weiterführende Informationen
- Bleibst sachlich und neutral

Formatiere alle Antworten in Markdown für optimale Lesbarkeit.
"""

# Testfragen
test_questions = [
    "Mein Lieblingsgericht ist Lasagne. Antworte kurz.",
    "Wie viele Tage hat eine Woche? Antworte kurz.",
    "Wie viele Tage hat ein Monat? Antworte kurz.",
    "Wie viele Tage hat ein Jahr? Antworte kurz.",
    "Was ist mein Lieblingsgericht?"
]

In [ ]:
# Abschnitt 3: Chat-Komponenten initialisieren
llm = ChatOpenAI(
    temperature=TEMPERATURE,
    model=MODEL
)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

def create_chain_input(input_dict: dict[str]) -> dict[str]:
    """Erstellt den Input für die Chain mit History."""
    return {
        "question": input_dict["question"],
        "system_prompt": DEFAULT_SYSTEM,
        "chat_history": input_dict["history"]
    }

# LCEL Chain mit History erstellen
chat_chain = (
    RunnableLambda(create_chain_input)
    | chat_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Abschnitt 4: Funktionen definieren
def format_history(history: list) -> list:
    """  Formatiert den Konversationsverlauf. """
    return history[-MEMORY_WINDOW:]

def interact_with_ai(question: str, history: list) -> list:
    """ Verarbeitet eine Konversation mit der KI. """
    # Nutzereingabe anzeigen
    display(Markdown(f"### 🧑‍🦱 Mensch:"))
    display(Markdown(question))

    # Antwort mit History generieren
    response = chat_chain.invoke({
        "question": question,
        "history": format_history(history)
    })

    # Antwort anzeigen
    display(Markdown(f"### 🤖 KI:"))
    display(Markdown(response))
    print()

    # Konversationsverlauf aktualisieren
    return history + [
        HumanMessage(content=question),
        AIMessage(content=response)
    ]

In [ ]:
# Abschnitt 5: Hauptprogramm

# Konversationsverlauf initialisieren
history = [SystemMessage(content=DEFAULT_SYSTEM)]

# Testfragen durchlaufen
for question in test_questions:
    history = interact_with_ai(question, history)

Bei diesem Ansatz speichern wir die gesamte Konversationshistorie in einer Liste und reichen sie bei jeder neuen Anfrage mit. Vorteil ist die volle Kontrolle und der Zugriff auf den gesamten Kontext. Der Nachteil: bei langen Unterhaltungen kann der Kontext die Token-Limits des Modells überschreiten.



<p><font color='black' size="5">
Framework-Komponenten
</font></p>



Als Alternative zur manuellen Verwaltung bietet LangChain spezialisierte Komponenten zur Konversationsverwaltung. Diese automatisieren viele der manuellen Schritte und bieten eine strukturiertere Herangehensweise.

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"
TEMPERATURE = 0.0
DEFAULT_SYSTEM = """
Du bist ein kompetenter KI-Assistent mit breitem Fachwissen.
Deine Antworten sind:
- Klar strukturiert und mit Markdown formatiert
- Praxisorientiert und direkt umsetzbar
- Basierend auf aktuellem Kenntnisstand
- Mit passenden Beispielen versehen
- In verständlicher Sprache formuliert
Bei deiner Arbeit:
- Analysierst du Fragen sorgfältig
- Gibst präzise und relevante Antworten
- Erkennst den Kontext der Anfrage
- Bietest bei Bedarf weiterführende Informationen
- Bleibst sachlich und neutral
Formatiere alle Antworten in Markdown für optimale Lesbarkeit.
"""

In [ ]:
# Abschnitt 3: Historienverwaltung und Chat-Komponenten initialisieren
# Dictionary für Sitzungs-Historien
session_histories = {}

# Hilfsfunktion zur Verwaltung der ChatMessageHistory
def get_session_history(session_id):
    if session_id not in session_histories:
        # Erstelle eine neue History, wenn keine existiert
        history = ChatMessageHistory()
        # Füge die System-Nachricht zur History hinzu
        history.add_message(SystemMessage(content=DEFAULT_SYSTEM))
        session_histories[session_id] = history
    return session_histories[session_id]

# LLM-Instanz initialisieren
llm = ChatOpenAI(
    temperature=TEMPERATURE,
    model=MODEL
)

# Chat-Prompt mit Platzhalter für die Historie
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", DEFAULT_SYSTEM),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# Basis-Chain ohne Historie erstellen
chain = chat_prompt | llm | StrOutputParser()

# Chain mit Message-History-Funktionalität erweitern
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # Unsere verbesserte History-Management-Funktion
    input_messages_key="question",
    history_messages_key="history"
)

In [ ]:
# Abschnitt 4: Funktionen definieren
def interact_with_ai(question: str, session_id: str = "kontinuierliche_session") -> str:
    """Verarbeitet eine Konversation und gibt die Antwort zurück."""
    # Nutzereingabe anzeigen
    display(Markdown(f"### 🧑‍🦱 Mensch:"))
    display(Markdown(question))

    # Antwort mit History generieren
    response = chain_with_history.invoke(
        {"question": question},
        config={"configurable": {"session_id": session_id}}
    )

    # Debug-Ausgabe: Zeige die aktuelle History an
    history = session_histories[session_id]

    # Antwort anzeigen
    display(Markdown(f"### 🤖 KI:"))
    display(Markdown(response))
    print()

    return response

In [ ]:
# Abschnitt 5: Hauptprogramm
# Konstante Session-ID definieren
KONSTANTE_SESSION_ID = "kontinuierliche_session"

# Testfragen
test_questions = [
    "Mein Lieblingsgericht ist Lasagne. Antworte kurz.",
    "Wie viele Tage hat eine Woche? Antworte kurz.",
    "Wie viele Tage hat ein Monat? Antworte kurz.",
    "Wie viele Tage hat ein Jahr? Antworte kurz.",
    "Was ist mein Lieblingsgericht?"
]

# Testfragen durchlaufen mit konstanter Session-ID
for question in test_questions:
    interact_with_ai(question, session_id=KONSTANTE_SESSION_ID)

In [ ]:
# Optional: Ausgabe der vollständigen Konversation am Ende
def zeige_konversation(session_id):
    """ Zeigt die vollständige Konversationshistorie für eine bestimmte Session an. """
    markdown_text = "## Vollständige Konversation\n"

    if session_id in session_histories:
        for i, msg in enumerate(session_histories[session_id].messages):
            # Bestimme die Rolle der Nachricht
            if isinstance(msg, SystemMessage):
                rolle = "⚙️ System"
            elif isinstance(msg, HumanMessage):
                rolle = "🧑‍🦱 Mensch"
            else:
                rolle = "🤖 KI"

            # Kürze den Inhalt, wenn er zu lang ist
            inhalt = msg.content
            if len(inhalt) > 50:
                inhalt = f"{inhalt[:50]}..."

            # Füge Nachricht zum Markdown-Text hinzu
            markdown_text += f"**{i}.** [{rolle}]: {inhalt}\n\n"
    else:
        markdown_text += "*Keine Konversationshistorie für diese Session gefunden.*\n\n"

    # Zeige den Markdown-Text an
    display(Markdown(markdown_text))

# Beispielaufruf
zeige_konversation(KONSTANTE_SESSION_ID)

Dieser Ansatz nutzt `RunnableWithMessageHistory` und `ChatMessageHistory` von LangChain, um Konversationen zu verwalten. Vorteile sind die einfachere Handhabung und die Möglichkeit, mehrere Konversationen mit unterschiedlichen Session-IDs zu verwalten.



# 2 | Nachrichten-Limit
---




Manchmal möchten wir die Anzahl der Nachrichten begrenzen, die ein Modell als Kontext erhält, anstatt die komplette Historie zu verwenden. Dies kann sinnvoll sein, wenn:

1. Der jüngste Kontext am relevantesten ist
2. Wir Tokens sparen wollen
3. Wir eine fokussiertere Antwort ohne zu viel Ablenkung durch ältere Themen wünschen



<p><font color='black' size="5">
ConversationBufferWindowMemory
</font></p>

In [ ]:
# Abschnitt 1: Importe
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from IPython.display import display, Markdown
from operator import itemgetter

In [ ]:
# Abschnitt 2: Konstanten
MODEL = "gpt-4o-mini"
TEMPERATURE = 0.0
DEFAULT_SYSTEM = "Du bist ein hilfsbereiter KI-Assistent."
MAX_HISTORY = 3  # Anzahl der zu speichernden Nachrichtenpaare

TEST_QUESTIONS = [
    "Mein Lieblingsgericht ist Lasagne. Antworte kurz.",
    "Wie viele Tage hat eine Woche? Antworte kurz.",
    "Wie viele Tage hat ein Monat? Antworte kurz.",
    "Wie viele Tage hat ein Jahr? Antworte kurz.",
    "Was ist mein Lieblingsgericht?"
]

In [ ]:
# Abschnitt 3: Chat-Komponenten initialisieren
llm = ChatOpenAI(temperature=TEMPERATURE, model=MODEL)

# Memory mit k=3 für Fenstergröße
memory = ConversationBufferWindowMemory(
    k=MAX_HISTORY,
    return_messages=True,
    memory_key="history"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", DEFAULT_SYSTEM),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

def get_chat_history(input_dict: dict) -> dict:
    """Lädt die Chat-Historie aus dem Memory."""
    memory_vars = memory.load_memory_variables({})
    return {
        "history": memory_vars["history"],
        "input": input_dict["input"]
    }

# Chain mit Pipe erstellen
chain = get_chat_history | prompt | llm

In [ ]:
# Abschnitt 4: Funktionen definieren
def interact_with_ai(question):
    """Führt eine Interaktion mit der KI durch und verarbeitet die Antwort."""
    # Ausgabe der Frage
    display(Markdown(f"\n🧑‍🦱 Mensch:"))
    display(Markdown(question))
    display(Markdown("\n🤖 KI: "))

    # KI-Antwort einholen
    response = chain.invoke({"input": question})

    # Antworttext extrahieren
    response_text = response.content if isinstance(response, AIMessage) else str(response)

    # Formatierte Ausgabe erstellen
    display(Markdown(response_text))
    print()

    # Konversation im Memory speichern
    memory.save_context({"input": question}, {"output": response_text})

    return response_text

In [ ]:
# Abschnitt 5: Hauptprogramm

for question in TEST_QUESTIONS:
    interact_with_ai(question)

Nach der Ausführung können wir den gespeicherten Kontext untersuchen:

In [ ]:
print("\nGespeicherter Kontext:")
memory_ai = memory.load_memory_variables({})["history"]
for i in range(len(memory_ai)):
    print(memory_ai[i].content)

Der Parameter `k` in `ConversationBufferWindowMemory` steuert, wie viele Nachrichten-Paare (Frage und Antwort) dem LLM als Kontext übergeben werden. Mit `k=3` werden nur die letzten drei Konversationsrunden berücksichtigt, auch wenn die gesamte Konversation länger ist.




<p><font color='black' size="5">
Memory-Funktion mit LCEL
</font></p>

Alternativ können wir auch unsere eigene Fenster-Implementierung erstellen:

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown

from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.schema import AIMessage

In [ ]:
# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"
MAX_HISTORY = 3  # Maximale Anzahl an gespeicherten Konversationsbeiträgen

DEFAULT_SYSTEM = """Du bist ein kompetenter KI-Assistent mit breitem Fachwissen.

Deine Antworten sind:
- Klar strukturiert und mit Markdown formatiert
- Praxisorientiert und direkt umsetzbar
- Basierend auf aktuellem Kenntnisstand
- Mit passenden Beispielen versehen
- In verständlicher Sprache formuliert

Aktuelle Konversation:
{history}
Mensch: {input}
KI:"""

TEST_QUESTIONS = [
    "Mein Lieblingsgericht ist Lasagne. Antworte kurz.",
    "Wie viele Tage hat eine Woche? Antworte kurz.",
    "Wie viele Tage hat ein Monat? Antworte kurz.",
    "Wie viele Tage hat ein Jahr? Antworte kurz.",
    "Was ist mein Lieblingsgericht?"
]

In [ ]:
# Abschnitt 3: Funktionen
# Speicher für Konversationshistorie manuell verwalten
conversation_history: list[str] = []

def update_memory(question: str, response: str) -> list[str]:
    """Fügt neue Konversationsbeiträge zur Historie hinzu."""
    global conversation_history
    conversation_history.append(f"Mensch: {question}\nKI: {response}")

    # Begrenze die Historie auf die letzten N Einträge
    if len(conversation_history) > MAX_HISTORY:
        conversation_history = conversation_history[-MAX_HISTORY:]

    return conversation_history

def format_history(history: list[str]) -> str:
    """Formatiert die Historie für die Eingabe an das Modell."""
    return "\n".join(history)

# Interaktionen mit dem Modell
def interact_with_ai(question):
    """Führt eine einzelne Interaktion mit der KI durch."""
    # Ausgabe der Frage
    display(Markdown(f"\n🧑‍🦱 Mensch:"))
    display(Markdown(question))
    display(Markdown("\n🤖 KI: "))

    # KI-Antwort einholen
    response = chain.invoke({"input": question})

    # Antworttext extrahieren
    response_text = response.content if isinstance(response, AIMessage) else str(response)

    # Formatierte Ausgabe erstellen
    display(Markdown(response_text))
    print()

    # Speicher aktualisieren
    update_memory(question, response_text)

    return response_text

In [ ]:
# Abschnitt 4: LLM und LCEL-Chain initialisieren
llm = ChatOpenAI(
    model_name=MODEL,
    temperature=0.0
)

prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=DEFAULT_SYSTEM
)

# RunnableMap für Input-Transformation
prepare_input = RunnableLambda(lambda x: {
    "history": format_history(conversation_history),
    "input": x["input"]
})

# LCEL-Chain erstellen
chain = prepare_input | prompt | llm

In [ ]:
# Abschnitt 5: Hauptprogramm

for question in TEST_QUESTIONS:
    interact_with_ai(question)

Diese Implementierung nutzt eine einfache Liste, um die letzten N Konversationspaare zu speichern. Sie bietet mehr Kontrolle, erfordert jedoch auch mehr manuellen Code.



# 3 | Token-Limit
---




Die vorherigen Ansätze begrenzten die Anzahl der gespeicherten Nachrichten. In der Praxis ist jedoch das **Token-Limit** des Modells entscheidend. `ConversationTokenBufferMemory` adressiert dieses Problem, indem es den Kontext basierend auf der tatsächlichen Token-Anzahl begrenzt.

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown
from langchain.chains import ConversationChain
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.schema import AIMessage

In [ ]:
# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"
MAX_TOKEN_LIMIT = 1024
DEFAULT_SYSTEM = """
Du bist ein kompetenter KI-Assistent mit breitem Fachwissen.

Deine Antworten sind:
- Klar strukturiert und mit Markdown formatiert
- Praxisorientiert und direkt umsetzbar
- Basierend auf aktuellem Kenntnisstand
- Mit passenden Beispielen versehen
- In verständlicher Sprache formuliert

Aktuelle Konversation:
{history}
Mensch: {input}
KI:
"""

In [ ]:
# Abschnitt 3: Funktionen definieren
def interact_with_ai(question):
    """ Führt eine Interaktion mit der KI durch und verarbeitet die Antwort. """
    # Zeige die Frage an
    display(Markdown(f"**🧑‍🦱 Mensch:**"))
    display(Markdown(f"{question}"))

    # KI-Antwort abrufen
    response = chain.invoke({"input": question})

    # Antwort je nach Typ verarbeiten
    if isinstance(response, dict) and "response" in response:
        response_text = response["response"]
    elif isinstance(response, AIMessage):
        response_text = response.content
    else:
        response_text = str(response)

    # Antwort anzeigen
    display(Markdown(f"**🤖 KI:**"))
    display(Markdown(f"{response_text}"))

    # Konversation im Memory speichern
    memory.save_context({"input": question}, {"output": response_text})

    # Kein return, um doppelte Ausgabe zu vermeiden
    return None

In [ ]:
# Abschnitt 4: LLM und Kette initialisieren
llm = ChatOpenAI(
    model_name=MODEL,
    temperature=0.0
)

memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=MAX_TOKEN_LIMIT,
    return_messages=True
)

# Prompt Template erstellen
prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=DEFAULT_SYSTEM
)

# Conversation Chain erstellen
chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=False
)

In [ ]:
# Abschnitt 5: Interaktionen mit dem Modell
interact_with_ai("Mein Name ist Ralf")
interact_with_ai("Warum ist der Himmel blau?")
interact_with_ai("Und warum ist er manchmal rot?")
interact_with_ai("Wie ist mein Name?")

Gespeicherten Kontext nach Ausführung:

In [ ]:
# Gespeicherter Kontext anzeigen
display(Markdown("## ✨ Gespeicherter Kontext:"))
display(Markdown("---"))

# Lade den Speicher
memory_ai = memory.load_memory_variables({})["history"]

# Erstelle eine formatierte Markdown-Ausgabe für alle Nachrichten
for msg in memory_ai:
    markdown_content = f"**Typ:** {msg.type}\n\n**Inhalt:** {msg.content}"
    display(Markdown(markdown_content))

Der Hauptvorteil dieses Ansatzes ist die präzise Kontrolle über die tatsächliche Token-Anzahl, die ans Modell übergeben wird. Statt willkürlich die letzten N Nachrichten zu speichern, behält diese Implementierung so viele Nachrichten wie möglich innerhalb des Token-Limits bei.



# 4 | Zusammenfassung
---




Die bisher betrachteten Ansätze haben entweder die vollständige Historie oder einen Teil davon bewahrt. Eine Alternative ist, längere Konversationen zusammenzufassen, um wichtige Informationen zu behalten, ohne die Token-Limits zu überschreiten.

LangChain's `ConversationSummaryMemory` speichert und aktualisiert eine Zusammenfassung der Konversation, anstatt ältere Chatverläufe zu vergessen. Dies ermöglicht die Bewahrung wichtiger Informationen und hält den Speicher effizient.

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain_core.runnables import RunnablePassthrough

# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"

conversations = [
    "Mein Name ist Ralf",
    "Warum ist der Himmel blau?",
    "Und warum ist er manchmal rot?",
    "Wie ist mein Name?"
]

# Abschnitt 3: Funktionen definieren
def interact_with_ai(question):
    """Führt eine einzelne Interaktion mit der KI durch und verarbeitet die Antwort."""
    display(Markdown(f"**🧑‍🦱 Mensch:**"))
    display(Markdown(f"{question}"))

    response = chain.invoke({"input": question})

    # Antwort extrahieren
    if isinstance(response, dict) and "response" in response:
        response_text = response["response"]
    elif isinstance(response, AIMessage):
        response_text = response.content
    else:
        response_text = str(response)

    display(Markdown(f"**🤖 KI:**"))
    display(Markdown(f"{response_text}"))
    print()

    # Speichere die Konversation im Memory
    memory.save_context({"input": question}, {"output": response_text})

    return response_text

# Abschnitt 4: Chain und Memory initialisieren
# LLM initialisieren
llm = ChatOpenAI(
    temperature=0,
    model_name=MODEL
)

# Memory initialisieren
memory = ConversationSummaryMemory(
    llm=llm,
    return_messages=True,
    max_token_limit=2000
)

# Prompt Template erstellen
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent. Nutze vorherige Konversationen für kontextbezogene Antworten."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Chain erstellen
chain = (
    RunnablePassthrough.assign(
        history=lambda x: memory.load_memory_variables({})["history"]
    )
    | prompt
    | llm
)

# Abschnitt 5: Nachrichten verarbeiten und Ergebnisse anzeigen
for user_input in conversations:
    interact_with_ai(user_input)

Nach der Ausführung können wir die erzeugte Zusammenfassung betrachten:

In [ ]:
# Ausgabe der Zusammenfassung mit display(Markdown())
display(Markdown("## Aktuelle Zusammenfassung:"))
summary_ai = memory.load_memory_variables({})["history"]
display(Markdown(f"{summary_ai[0].content}"))

Dieser Ansatz ist besonders nützlich für längere Konversationen, da er die wichtigsten Informationen extrahiert und kompakt behält, statt nur die letzten N Nachrichten. Dadurch bleibt der Kontext erhalten, ohne das Token-Limit zu sprengen.

# 5 | Persistenter Speicher
---




Die Möglichkeit eines ChatBots, sich an vorherige Unterhaltungen zu erinnern, spielt eine zentrale Rolle für eine flüssige und konsistente Kommunikation. Ein **persistenter Speicher** erlaubt es, Gesprächsdaten über mehrere Sitzungen hinweg zu sichern und wiederzuverwenden. Dadurch kann der ChatBot nicht nur den Kontext besser erfassen, sondern auch langfristige Nutzerpräferenzen berücksichtigen.

Dieser Abschnitt untersucht verschiedene Speicherlösungen für ChatBots und deren effiziente Integration. Dabei werden unterschiedliche Methoden analysiert – von einfachen Dateisystemen über relationale und NoSQL-Datenbanken bis hin zu spezialisierten Speichern für große Sprachmodelle. Zudem wird aufgezeigt, wie gespeicherte Informationen verwaltet, aktualisiert und geschützt werden, um eine ausgewogene Kombination aus Leistungsfähigkeit, Datenschutz und Benutzerfreundlichkeit zu gewährleisten.

Die Implementierung eines dauerhaften Speichers ermöglicht es einem ChatBot, nicht nur die Interaktion mit Nutzern zu verbessern, sondern auch personalisierte, kontextbezogene und langfristig relevante Dialoge zu führen.

Für die nachfolgende Version wird die Klasse **BaseMemory** für einen persistenten Chat verwendet. Der Chat-Verlauf wird in einer JSON-Datei gespeichert. Diese Klasse dient als abstrakte Basisklasse, von der spezifische Memory-Implementierungen abgeleitet werden, um Kontexte und Zwischenergebnisse in den Arbeitsabläufen (Chains) zu speichern und wiederzuverwenden.


<p><font color='black' size="5">
Minimalversion mit History als .txt
</font></p>

In [ ]:
import os
from openai import OpenAI

def chat():
    # Initialisierung des OpenAI-Clients
    client = OpenAI()
    file = "chat_history.txt"

    print("Chatbot gestartet.")
    print("Befehle: 'history' zeigt Verlauf, 'exit' beendet Chat")

    # Chat-Kontext initialisieren
    messages = [
        {"role": "system", "content": "Du bist ein hilfreicher Assistent. Antworte auf Deutsch und sei präzise."}
    ]

    # Lade bestehende Historie beim Start, falls vorhanden
    try:
        if os.path.exists(file) and os.path.getsize(file) > 0:
            print("Lade vorherige Chathistorie...")
            chat_pairs = []
            with open(file, 'r', encoding='utf-8') as f:
                content = f.read()
                segments = content.split("\n\n")

                for segment in segments:
                    if segment.strip() and "Benutzer: " in segment and "KI: " in segment:
                        parts = segment.split("KI: ", 1)
                        if len(parts) == 2:
                            user_part = parts[0].replace("Benutzer: ", "", 1).strip()
                            ai_part = parts[1].strip()
                            if user_part and ai_part:
                                chat_pairs.append((user_part, ai_part))

            # Füge die letzten 5 Austausche zum Kontext hinzu (falls vorhanden)
            for user_msg, ai_msg in chat_pairs[-10:]:
                messages.append({"role": "user", "content": user_msg})
                messages.append({"role": "assistant", "content": ai_msg})

            print(f"{len(chat_pairs)} frühere Unterhaltungen gefunden, die letzten {min(10, len(chat_pairs))} werden im Kontext verwendet.")
    except Exception as e:
        print(f"Fehler beim Laden der Historie: {e}")

    # Endlosschleife für den Chat
    while True:
        user_input = input("\nBenutzer: ")

        if user_input == "exit":
            print("Chat beendet.")
            break
        elif user_input == "history":
            try:
                with open(file, 'r', encoding='utf-8') as f:
                    print("\n--- Chatverlauf ---")
                    print(f.read())
                    print("--- Ende des Verlaufs ---\n")
            except FileNotFoundError:
                print("Keine Chathistorie gefunden.")
            continue

        # Benutzer-Nachricht zum Kontext hinzufügen
        messages.append({"role": "user", "content": user_input})

        # OpenAI API-Anfrage mit vollem Kontext
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=1000
            )
            ai_response = response.choices[0].message.content

            # Antwort zum Kontext hinzufügen für zukünftige Anfragen
            messages.append({"role": "assistant", "content": ai_response})

            # Kontext begrenzen, um Token zu sparen (behält nur die letzten 10 Nachrichten + System-Prompt)
            if len(messages) > 11:  # System-Prompt + 10 Nachrichten
                messages = [messages[0]] + messages[-10:]

        except Exception as e:
            ai_response = f"Fehler: {str(e)}"

        print(f"KI: {ai_response}")

        # Speichern in einfacher Textdatei
        with open(file, 'a', encoding='utf-8') as f:
            f.write(f"Benutzer: {user_input}\nKI: {ai_response}\n\n")

In [ ]:
chat()

<p><font color='black' size="5">
Komfortversion mit History als .json
</font></p>

In [ ]:
# Abschnitt 1: Importe
import json
import os
from datetime import datetime
from pathlib import Path
from langchain_core.memory import BaseMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory
from pydantic import BaseModel, Field

from IPython.display import display, Markdown

In [ ]:
# Abschnitt 2: Konstanten und Klassen
MODEL = "gpt-4o-mini"
DEFAULT_MEMORY_FILE = "chat_memory.json"
SYSTEM_PROMPT = """Du bist ein kompetenter KI-Assistent mit breitem Fachwissen.

Deine Antworten sind:
- Klar strukturiert und mit Markdown formatiert
- Praxisorientiert und direkt umsetzbar
- Basierend auf aktuellem Kenntnisstand
- Mit passenden Beispielen versehen
- In verständlicher Sprache formuliert"""

class EnhancedJSONMemory(BaseMemory, BaseModel):
    """Erweiterte Memory-Implementierung mit JSON-Speicherung, Statistiken und Zusammenfassung."""
    file_path: str = Field(default=DEFAULT_MEMORY_FILE)
    summary_memory: object = Field(default=None)
    chat_data: dict[str, object] = Field(default_factory=lambda: {
        "conversations": [],
        "metadata": {
            "created_at": datetime.now().isoformat(),
            "last_modified": datetime.now().isoformat(),
            "total_interactions": 0,
            "model_name": MODEL,
            "version": "2.0"
        },
        "statistics": {
            "average_response_length": 0,
            "total_chars_exchanged": 0
        },
        "summary": ""
    })

    def __init__(self, **kwargs) -> None:
        """Initialisiert die Memory-Klasse und lädt existierende Chatverläufe."""
        super().__init__(**kwargs)
        self.summary_memory = ConversationSummaryBufferMemory(
            llm=ChatOpenAI(temperature=0, model=MODEL),
            max_token_limit=2000,
            memory_key="summary",
            human_prefix="Mensch",
            ai_prefix="KI"
        )
        self.load_history()

    def format_timestamp(self, timestamp_str: str) -> str:
        """Formatiert einen ISO-Zeitstempel in ein lesbares Format."""
        return datetime.fromisoformat(timestamp_str).strftime("%d.%m.%Y %H:%M:%S")

    def update_statistics(self, user_input: str, response: str) -> None:
        """Aktualisiert die Chatstatistiken."""
        stats = self.chat_data["statistics"]
        total_responses = len(self.chat_data["conversations"])

        if total_responses > 0:
            current_total = stats["average_response_length"] * (total_responses - 1)
            stats["average_response_length"] = round((current_total + len(response)) / total_responses, 2)

        stats["total_chars_exchanged"] += len(user_input) + len(response)

    def update_summary(self, user_input: str, response: str) -> None:
        """Aktualisiert die Konversationszusammenfassung."""
        self.summary_memory.save_context({"input": user_input}, {"output": response})
        self.chat_data["summary"] = self.summary_memory.load_memory_variables({}).get("summary", "")

    def save_context(self, inputs: dict[str], outputs: dict[str, str]) -> None:
        """Speichert den aktuellen Kontext mit erweiterten Metadaten."""
        self.chat_data["conversations"].append({
            "timestamp": datetime.now().isoformat(),
            "user_input": inputs["input"],
            "response": outputs["text"],
            "interaction_id": len(self.chat_data["conversations"]) + 1
        })
        self.chat_data["metadata"]["last_modified"] = datetime.now().isoformat()
        self.chat_data["metadata"]["total_interactions"] += 1

        self.update_statistics(inputs["input"], outputs["text"])
        self.update_summary(inputs["input"], outputs["text"])
        self._save_to_file()

    def _save_to_file(self) -> None:
        """Speichert den Chatverlauf in der JSON-Datei."""
        try:
            with Path(self.file_path).open("w", encoding="utf-8") as file:
                json.dump(self.chat_data, file, ensure_ascii=False, indent=2)
        except Exception as e:
            md_print(f"⚠️ **Fehler beim Speichern der Historie:** {str(e)}")

    def load_history(self) -> None:
        """Lädt die Chathistorie aus der JSON-Datei."""
        if not Path(self.file_path).exists():
            return

        try:
            with Path(self.file_path).open("r", encoding="utf-8") as file:
                loaded_data = json.load(file)

            if "metadata" in loaded_data and "version" in loaded_data["metadata"]:
                self.chat_data = loaded_data
                md_print("✅ **Konversationshistorie wurde erfolgreich geladen.**")

                # Lade vorhandene Konversationen in das Summary Memory
                for conv in self.chat_data["conversations"]:
                    self.summary_memory.save_context(
                        {"input": conv["user_input"]},
                        {"output": conv["response"]}
                    )
            else:
                md_print("⚠️ **Veraltetes Historienformat erkannt. Erstelle neue Historie.**")
        except json.JSONDecodeError as e:
            md_print(f"⚠️ **Fehler beim Laden der JSON-Datei:** {str(e)}")

    def load_memory_variables(self, inputs: dict[str]) -> dict[str, list[dict[str]]]:
        """Lädt die Chatvariablen für den Prompt."""
        return {"history": self.chat_data["conversations"], "summary": self.chat_data["summary"]}

    @property
    def memory_variables(self) -> list[str]:
        """Definiert die verfügbaren Memory-Variablen."""
        return ["history", "summary"]

    def clear(self) -> None:
        """Löscht die Chathistorie und speichert den zurückgesetzten Zustand."""
        # Zurücksetzen des Konversations-Arrays
        self.chat_data["conversations"] = []
        # Zurücksetzen der Zusammenfassung
        self.chat_data["summary"] = ""
        # Zurücksetzen der Statistiken
        self.chat_data["statistics"] = {
            "average_response_length": 0,
            "total_chars_exchanged": 0
        }
        # Metadaten aktualisieren
        self.chat_data["metadata"]["last_modified"] = datetime.now().isoformat()
        self.chat_data["metadata"]["total_interactions"] = 0

        # Summary Memory zurücksetzen
        self.summary_memory.clear()

        # Gespeicherte Datei aktualisieren
        self._save_to_file()

        md_print("🧹 **Chathistorie wurde erfolgreich gelöscht.**")

    def show_history(self) -> None:
        """Zeigt eine detaillierte Konversationshistorie und Zusammenfassung."""
        if not self.chat_data["conversations"]:
            md_print("❌ **Keine Konversationshistorie vorhanden.**")
            return

        history_markdown = []
        history_markdown.append("\n## 📊 Chat-Statistiken:")
        stats = self.chat_data["statistics"]
        meta = self.chat_data["metadata"]
        history_markdown.append(f"- **Gesamtinteraktionen:** {meta['total_interactions']}")
        history_markdown.append(f"- **Durchschnittliche Antwortlänge:** {stats['average_response_length']} Zeichen")
        history_markdown.append(f"- **Gesamter Zeichenaustausch:** {stats['total_chars_exchanged']} Zeichen")

        if self.chat_data["summary"]:
            history_markdown.append("\n## 📝 Zusammenfassung der Konversation:")
            history_markdown.append(self.chat_data["summary"])

        history_markdown.append("\n## 📜 Detaillierte Konversationshistorie:")
        for conv in self.chat_data["conversations"]:
            history_markdown.append("\n" + "="*50)
            history_markdown.append(f"### 🔢 Interaktion #{conv['interaction_id']}")
            history_markdown.append(f"**🕒 Zeitpunkt:** {self.format_timestamp(conv['timestamp'])}")
            history_markdown.append(f"**🧑‍🦱 Mensch:** {conv['user_input']}")
            history_markdown.append(f"**🤖 KI:**\n{conv['response']}")

        md_print("\n".join(history_markdown))


In [ ]:
# Abschnitt 3: Chat Komponenten
# LLM und Memory initialisieren
llm = ChatOpenAI(model=MODEL, temperature=0.0)
memory = EnhancedJSONMemory()

# Prompt Template und Chat Sequence erstellen
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=SYSTEM_PROMPT),
    ("system", "Zusammenfassung der bisherigen Konversation:\n{summary}\n"),
    ("system", "Bisheriger Chatverlauf:\n{history}\n"),
    ("human", "{input}")
])

# Hilfsfunktion für Formatierung
def format_history(history):
    return "\n".join([f"Mensch: {entry['user_input']}\nKI: {entry['response']}" for entry in history])

# Chat Sequence erstellen
chat_sequence = (
    {
        "history": lambda x: format_history(memory.load_memory_variables({})["history"]),
        "summary": lambda x: memory.load_memory_variables({})["summary"],
        "input": lambda x: x["input"]
    }
    | prompt
    | llm
)

In [ ]:
# Abschnitt 4: Funktionen
def md_print(text):
    """Gibt Text als Markdown aus, wenn möglich, sonst als normalen Text."""
    try:
        # Prüfen, ob wir in einem Jupyter Notebook sind
        is_notebook = 'ipykernel' in sys.modules
        if is_notebook:
            display(Markdown(text))
        else:
            print(text)
    except NameError:
        # Wenn sys nicht definiert ist, nehmen wir an, dass wir nicht in einem Notebook sind
        print(text)

def run_chat(chat_sequence, memory) -> None:
    """Führt die Chat-Schleife aus."""
    md_print("## Chat gestartet\n")
    md_print("**Befehle:**\n- `history`: Zeigt die Chathistorie\n- `clear`: Löscht die Chathistorie\n- `exit` oder `tschüss`: Beendet den Chat\n")

    # Prüfen, ob wir in einem Jupyter Notebook sind
    try:
        is_notebook = 'ipykernel' in sys.modules
    except NameError:
        # sys ist nicht definiert, also sind wir nicht in einem Notebook
        is_notebook = False

    while True:
        # Die Eingabeaufforderung bleibt ein normales print, da input() damit arbeitet
        if is_notebook:
            from IPython.display import clear_output
            # In Notebooks können wir es schöner machen mit garantierter Breitenreduzierung
            from ipywidgets import widgets

            # Container-Widget mit fester Breite erstellen
            container = widgets.HBox(
                layout=widgets.Layout(
                    width='60%',       # Container auf 60% der verfügbaren Breite beschränken
                    margin='0px'
                )
            )

            # Textfeld erstellen (nimmt die volle Breite des Containers ein)
            user_input_widget = widgets.Text(
                description="🧠",
                placeholder="Deine Nachricht hier...",
                style={'description_width': '40px'},
                layout=widgets.Layout(width='100%')  # Füllt den Container vollständig aus
            )

            # Das Textfeld in den Container einfügen und anzeigen
            container.children = [user_input_widget]
            display(container)

            # Warten auf Eingabe
            user_input = input("Eingabe: ")
            user_input_widget.value = user_input
        else:
            user_input = input("\U0001F9B1 Mensch: ")

        if user_input.lower() in ["exit", "tschüss"]:
            break
        elif user_input.lower() == "history":
            memory.show_history()
            continue
        elif user_input.lower() == "clear":
            memory.clear()
            continue

        response = chat_sequence.invoke({"input": user_input})
        memory.save_context({"input": user_input}, {"text": response.content})

        # Markdown-Ausgabe für die Antwort
        md_print(f"\n## 🤖 KI:\n{response.content}\n")

In [ ]:
# Abschnitt 5: Hauptfunktion
def main() -> None:
    """Hauptfunktion zum Starten des Chat-Bots."""
    md_print("## 🤖 KI-Chat mit Gedächtnis")

    # sys importieren, wenn noch nicht geschehen
    try:
        import sys
    except ImportError:
        pass

    # Chat-Komponenten sind bereits global definiert
    run_chat(chat_sequence, memory)

In [ ]:
# Ausführen, wenn direkt gestartet
main()

# A | Aufgaben
---




Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
KI-gestütztes Notizbuch mit automatischer Kategorisierung
</font></p>


Entwickeln Sie ein KI-Notizbuch, das Eingaben von Nutzern speichert, automatisch kategorisiert (z. B. "Technologie", "Privat", "Aufgaben") und kontextbezogene Vorschläge liefert. Verwenden Sie dazu ein LangChain Memory-Konzept, z.B. ConversationBufferMemory, für eine effiziente Verwaltung.



<p><font color='black' size="5">
Chatbot: Nachrichten- vs. Summary-Speicher
</font></p>


Erstellen Sie einen einfachen Chatbot mit LangChain, der zwischen Nachrichten- und Summary-Speicher unterscheidet. Analysieren Sie, wie sich die Speichertypen auf die Qualität der Antworten auswirken.



<p><font color='black' size="5">
Automatische Protokollerstellung für Meetings
</font></p>


Entwickeln Sie ein System, das Gesprächsverläufe speichert und automatisch eine zusammenfassende Chat-Notiz erstellt. Nutzen Sie z.B. ConversationSummaryMemory, um die wichtigsten Punkte aus langen Gesprächen zu extrahieren.



<p><font color='black' size="5">
Virtuelle Assistenten mit eigenem Gedächtnis
</font></p>


Eine Datei wird bereitgestellt, die ein Gespräch zwischen zwei virtuellen Assistenten enthält. Jede Antwort soll aus genau einem Satz bestehen. Die Datei ist folgendermaßen aufgebaut:

|Assistent|Eingabe|
|---|---|
|botA|Ich heiße Max.|
|botB|Mein Name ist Emma.|
|botA|Ich wohne in Berlin.|
|botB|Ich lebe in Hamburg.|
|botA|Wo wohne ich und wie heiße ich?|
|botB|Wo lebe ich?|

**Aufgabe**  
Schreiben Sie ein Programm, das zwei virtuelle Assistenten simuliert, die jeweils ein eigenes Gedächtnis haben. Die Eingaben aus der Datei werden an den jeweiligen Assistenten geschickt, und ihre Antworten sollen in einer neuen Datei gespeichert werden.

**Erwartete Ausgabe**  
Die erwartete Ausgabe könnte wie folgt aussehen:

|Assistent|Antwort|
|---|---|
|botA|Hallo Max.|
|botB|Hallo Emma.|
|botA|Danke für diese Information.|
|botB|Danke für diese Information.|
|botA|Hallo Max, du wohnst in Berlin und dein Name ist Max.|
|botB|Hallo Emma, du lebst in Hamburg.|

**Hinweise**

- Die Antworten müssen nicht exakt mit der Beispielausgabe übereinstimmen, sollten aber sinngemäß ähnlich sein.
- Jeder Assistent hat ein eigenes Gedächtnis und sollte sich nur an seine eigenen Informationen erinnern.
- Das Programm soll die Antworten in einer neuen Datei im gleichen Tabellenformat speichern.